In [ ]:
# install botocore and boto3
# %pip install botocore boto3
# %pip install mypy-boto3-iam
%pip freeze > requirements.txt

In [ ]:
import boto3
import json

# Create IAM client
iam = boto3.client('iam')

# Create a role
role_name = 'my-test-role'
assume_role_policy_document = {
    'Version': '2012-10-17',
    'Statement': [
        {
            'Effect': 'Allow',
            'Principal': {
                'Service': 'ec2.amazonaws.com'
            },
            'Action': 'sts:AssumeRole'
        }
    ]
}

create_response = iam.create_role(
    RoleName=role_name,
    AssumeRolePolicyDocument=json.dumps(assume_role_policy_document)
)

print('Role ARN:', create_response['Role']['Arn'])

In [ ]:
delete_response = iam.delete_role(RoleName=role_name)
print('Role deleted, status:', delete_response['ResponseMetadata']['HTTPStatusCode'], 
      'and requestId: ', delete_response['ResponseMetadata']['RequestId'])

In [ ]:

from pathlib import Path
import json

print('Project Rootpath: ', rootpath.resolve())

trust_file_path=rootpath/Path('cicd-role/trust-relationship.json')
print('Trust File Path: ', trust_file_path.resolve())

with open(trust_file_path, 'r') as trust_file:
    trust_policy = json.load(trust_file)
    print(trust_policy)

# replace variables in trust policy


In [ ]:
from enum import Enum

class Environment(Enum):
    development = "dev"
    testplan = "tpe"
    experiment = "exp"
    production = "prd"

env_name = "development"

if isinstance(env_name, Environment):
    print(f"Environment: {env_name}")
else:
    print("Invalid Environment")

In [ ]:
from string import Template
from pathlib import Path
import json

rootpath = Path('../..')

def read_json_file_template(file_path: Path) -> Template:
    with open(file_path, 'r') as file:
        json_content=json.load(file)
        return Template(json.dumps(json_content))


In [ ]:
cdk_policy_path=rootpath/Path('cicd-role/policies/assume-cdk-policy.json')
cdk_policy=read_json_file_template(cdk_policy_path)
print("cdk policy template variables: ", cdk_policy.get_identifiers())

In [ ]:
cli_policy_path=rootpath/Path('cicd-role/policies/cli-policy.json')
cli_policy=read_json_file_template(cli_policy_path)
print("cli policy template variables: ", cli_policy.get_identifiers())

In [ ]:
inline_policies_dir = rootpath/Path('cicd-role/policies/inline')
for inline_policy_path in inline_policies_dir.rglob("*.json"):
    inline_policy_name=inline_policy_path.stem
    inline_policy_template=read_json_file_template(inline_policy_path)
    print(f"inline policy {inline_policy_name}, template variables: ", inline_policy_template.get_identifiers())
    # print("is template valid?", inline_policy_template.is_valid(), 'reason: ', [a for a in inline_policy_template.pattern.finditer(inline_policy_template.template)])
    print(inline_policy_template.safe_substitute({'aaws_principal_account': '123456789012'}))


In [ ]:
dist_dir = Path("dist")
print("dist dir: ", dist_dir.resolve()," isExists? ", dist_dir.exists())
text_file=dist_dir/"mytext.txt"
print("Text File Path:", text_file.resolve())
print([f.resolve() for f in dist_dir.rglob("*.txt")])
env_name="development"
env=Environment[env_name]
print(env_name, env, isinstance(env, Environment), isinstance(env_name, Environment))
# print(Environment[Environment[env]])
print("".join([s.capitalize() for s in "api-lambda-policy".split("-")]))

In [ ]:
import boto3

iam = boto3.client("iam")
def get_policy(policy_name: str):
    get_policy_response = iam.get_policy(
            PolicyArn=f"arn:aws:iam::471112824979:policy/{policy_name}")

    print(get_policy_response)
    return get_policy_response

In [ ]:
get_policy("CdkApiLambda")

get_policy("DummyCdkApiLambda")


In [ ]:
import enum

print(type(Environment))
print(isinstance(Environment, enum.EnumType))